In [1]:
%%writefile jetson_global_exec.py
import paramiko
import sys
import argparse
parser = argparse.ArgumentParser()
parser.add_argument("--command", type=str, default="ls", help="linux command to execute globally")
parser.add_argument("--min", type=str, default="20101")
parser.add_argument("--max", type=str, default="20136")
args = parser.parse_args()
JETSON_IP = "147.47.200.209"
MIN_PORT = args.min
MAX_PORT = args.max
USERNAME = "jetson"
PASSWORD = "jetson"
COMMAND = args.command

class GlobalExecute:
    def __init__(self):
        self.cli = paramiko.SSHClient()
        self.cli.set_missing_host_key_policy(paramiko.MissingHostKeyPolicy())
        
        self.clients = {}
        self.ping()

    def ping(self):
        self.available = []
        for port in range(int(MIN_PORT), int(MAX_PORT)+1):
            if port % 10 > 6 or port % 10 == 0:
                continue

            try:
                self.available.append(port)

                cli = paramiko.SSHClient()
                cli.set_missing_host_key_policy(paramiko.MissingHostKeyPolicy())
                cli.connect(JETSON_IP, port=port, username=USERNAME, password=PASSWORD)

                stdin, stdout, stderr = cli.exec_command("ls")
                lines = stdout.readlines()
            
                if lines:
                    self.clients[port] = cli
                    print(f"{port} is working: ls {lines}")

                
            except Exception as e:
                print(f"{port} {e}")
                continue
    
        temp = [i for i in range(int(MIN_PORT), int(MAX_PORT)+1)]
        result = []

        for port in temp:
            if port % 10 <= 6 and port % 10 > 0:
                result.append(port)

        print("NOT CONNECTED PORTS : ", list(set(result)-set(self.available)))

    def global_exec(self, command):
        # command : linux command to execute 
        print(f"Executes '{command}'")
        for port in self.available:
            try:
                cli = self.clients[port]
                stdin, stdout, stderr = cli.exec_command(f"{command}")
            
                if stdout:
                    stdout = stdout.readlines()
                if stderr:
                    stderr = stderr.readlines()
            
                print(f"Port {port} OUTPUT {stdout}")
                
                if stderr:
                    print(f"Port {port} ERROR {stderr}")
            

            except Exception as e:
                print(f"Port {port} error {e}")
                continue


    
if __name__ == "__main__":
    cl = GlobalExecute()
    cl.global_exec(COMMAND)
    
    res = "y"
    while res == "y":
        res = input("More commands? (y/n)\n")
        if res == "y":
            new_command = input("new command:\n")
            new_command.strip("\n")
            cl.global_exec(str(new_command))
        
        else:
            for k, v in cl.clients.items():
                v.close()
            sys.exit()

Overwriting jetson_global_exec.py


In [14]:
import os 
os.getcwd()

'C:\\Users\\david\\Desktop\\앰비언트\\Practice # - FL\\ambient_fl'

In [11]:
!python jetson_global_exec.py --command "ls"

Python


In [12]:
import paramiko
from scp import SCPClient

file_path = "../../../ambient_fl.zip"

def createSSHClient(server, port, user, password):
    client = paramiko.SSHClient()
    client.load_system_host_keys()
    client.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    client.connect(server, port, user, password)
    return client

MAX_PORT= 20136
MIN_PORT = 20101

for port in range(int(MIN_PORT), int(MAX_PORT)+1):
    if port % 10 > 6 or port % 10 == 0:
        continue
    
    try:
        ssh = createSSHClient("147.47.200.209", str(port), "jetson", "jetson")
        scp = SCPClient(ssh.get_transport())
        scp.put(file_path, ".")
        print(f"{port} done")
    
    except Exception as e:
        print(e)
        continue



20101 done
20102 done
20103 done
20104 done
20105 done
20106 done
20111 done
20112 done
20113 done
20114 done
20115 done
20116 done
20121 done
20122 done
20123 done
20124 done
20125 done
20126 done
20131 done
20132 done
20133 done
20134 done
20135 done
20136 done


In [24]:
!git add Guide.ipynb jetson_global_exec.py

The file will have its original line endings in your working directory


In [6]:
!git commit -m "distribute"

[master b9b59db] distribute
 1 file changed, 78 insertions(+), 19 deletions(-)


In [7]:
!git push origin master

To https://github.com/crazyboy9103/ambient_fl
   192ba5d..b9b59db  master -> master


In [ ]:
!python3 ./ambient_fl/FL_Client/jetson_client.py --ip "147.47.200.209" --p 9103 --max 5 --delay 5 --num 5